<a href="https://colab.research.google.com/github/zayar224/Lab/blob/master/GhostStory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import pandas as pd
import numpy as np
import string, os
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_excel('/content/GhostData.xlsx', encoding='UTF-8-sig')
all_story = sorted(list(set(df)))
len(all_story)
all_story

['label', 'story']

In [20]:
first = df['story']

In [21]:
sData = sorted(list(set(first)))
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}\t\n', lower=None, split=' ', char_level=False, oov_token=None, document_count=0)

def get_sequence_of_tokens(first):
  t.fit_on_texts(first)
  total_words = len(t.word_index) + 1

  input_sequences = []
  for line in first:
    token_list = t.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)
  
  return input_sequences, total_words
input_sequences, total_words = get_sequence_of_tokens(first)
input_sequences[:10]

[[445, 446],
 [445, 446, 447],
 [445, 446, 447, 448],
 [445, 446, 447, 448, 449],
 [445, 446, 447, 448, 449, 450],
 [445, 446, 447, 448, 449, 450, 43],
 [445, 446, 447, 448, 449, 450, 43, 451],
 [445, 446, 447, 448, 449, 450, 43, 451, 452],
 [445, 446, 447, 448, 449, 450, 43, 451, 452, 206],
 [445, 446, 447, 448, 449, 450, 43, 451, 452, 206, 207]]

In [22]:
##Pad Sequence And Create Predictions And Label
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len,padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
  label = ku.to_categorical(label, num_classes=total_words)

  return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

In [23]:
## Modeling
def create_model(max_sequence_len, total_words):
  model = Sequential()

  #Add Input Embedding Layer
  model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))

  #Add Hidden Layer 1 - LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))

  #Add Output Layer
  model.add(Dense(total_words, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam')

  return model

model = create_model(max_sequence_len, total_words)
model.summary()
model.fit(predictors,label, epochs=100, verbose=5)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 816, 10)           43570     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4357)              440057    
Total params: 528,027
Trainable params: 528,027
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20

In [26]:
## Generate trained text
def generate_text(seed_text, next_words, model, max_seq_len):
  for _ in range(next_words):
    token_list = t.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

    predicted = model.predict_classes(token_list, verbose=0)

    output_word = ''

    for word, index in t.word_index.items():
      if index == predicted:
        output_word = word
        break

    seed_text = seed_text + " " + output_word
  
  return seed_text.title()

In [28]:
print(generate_text("Ghost", 1000, model, max_sequence_len))

Ghost ညီမလေး တစ်ယောက် ပြောပြထားတာလေး ပြန်ရေးတင်ပေးပါ့မယ် Pageစ ကတည်းက Post တင်ရင် ပို့ပေးထားတဲ့ Ss လေးတွေ Cmt မှာ အမြဲပြပေးပါတယ် ပြီးခဲ့တဲ့ တစ်ပတ်လောက်ကတော့ မထည့်ပေးလည်း ရလောက်ပြီဆိုပြီး မထည့်ဘဲနေတော့ တချို့ မလိုလားတဲ့ အာရုံလေးတွေနောက်ရလို့ အခု Ssလေးတွေ Cmt မှာ ပြန်ပြထားပေးပါတော့မယ်နော် 🖤🦉 သူမမွေးခင် သူ့အမေ ကိုယ်ဝန်ဆောင် နေတဲ့ အချိန် မှာ ကြုံခဲ့ရတဲ့အကြာင်းလေးပါ တစ်ရက် ရွာက သူ့အဖွား အဖေဘက်က အသည်းအသန်ဖြစ်တယ်ဆိုတော့ သူ့အဖေ နဲ့ အမေလည်း ရွာကို အပြေးသွားရတော့တာပေါ့ ရောက်တော့ သူ့အဖွားအခြေအနေက တော်တော်ဆိုးနေပြီ ပြောရရင် နောက်ဆုံးအခြေအနေဖြစ်နေပြီပေါ့ အဲလိုနဲ့ တစ်နေ့ အဖွားက ဘေးမှာ ဘယ်သူမှမရှိတဲ့အချိန် အိမ်သာသွားချင်တော့ သူ့ဘာသာထသွားတယ် အဖွား အိပ်တဲ့ ကုတင်က နောက်ဖေးပေါက်နဲ့ ကပ်ရပ်ပဲဆိုတော့ အပေါက်အပြင်အထိတော့ တုန်ချိချိနဲ့ ရအောင်ထွက်လိုက်သေးတယ် အပြင်လည်း ရောက်ရော လဲပါလေရော အဲ့ကျမှ အိမ်သားတွေ က အသံကြားပြီး ရောက်လာတယ် အဖွားကိုထူပြီး အိပ်ရာထဲပြန်ပို့တာပေါ့ အဲ့ညက အရင်က တငြီးငြီးတညူညူနဲ့ ဖြစ်နေတဲ့အဖွားက ညဉ့်နက်ထိ အိမ်သားတွေနဲ့ထိုင် စကားပြောနေတယ် အကုန် လုံး နည်းနည်း တော့ အံ့သြကြတာပေါ့ ပြန်သက်သာလာပြီဆိုပ